In [40]:
import numpy as np
import re


palabras_a_indice = {}
indices_a_palabras = {}
diccionario_onehot = {}
diccionario_onehot_a_palabra = {}
diccionario_conteo = {}

with open("C:\\Users\\User\\Documents\\GitHub\\Aprendizaje_Automatico\\Evaluacion_Modelos\\corpus_junto2_todos_los_fuegos.txt", "r", encoding="utf-8") as f:
    words = f.read().splitlines()

'''def unir_palabras_en_contexto(corpus, palabra_objetivo1, palabra_objetivo2, contexto=1):
    corpus_modificado = []
    i = 0

    while i < len(corpus):
        palabra = corpus[i]

        # Si encontramos una de las palabras objetivo
        if palabra == palabra_objetivo1 or palabra == palabra_objetivo2:
            # Miramos dentro del rango del contexto (siguiente palabra incluida)
            for j in range(1, contexto + 1):
                if i + j < len(corpus):
                    siguiente = corpus[i + j]

                    if {palabra, siguiente} == {palabra_objetivo1, palabra_objetivo2}:
                        # Respetar orden según aparición
                        token = f"{palabra} {siguiente}"
                        corpus_modificado.append(token)
                        i += j + 1  
                        break
            else:
                
                corpus_modificado.append(palabra)
                i += 1
        else:
            corpus_modificado.append(palabra)
            i += 1

    return corpus_modificado

palabras_a_unir = [
                    (',','y'),
                    ('.','no'),
                    ('. ','no'),
                    ('que', 'lo'),
                    ('de','la'),
                    ('a','la'),
                    ('en','el'),
                    ('pero',','),
                    ('lo','que'),
                    ('la','de'),
                    ('con',','),
                    ('se','que'),
                    ('como', 'un'),
                    ('por','la'),
                    ('o',','),
                    ('no','?'),
                    ('para','que'),
                    (';','el'),
                    ('la',':')
]

corpus_modificado = words.copy()

# Función para unir las palabras en contexto
for palabra1, palabra2 in palabras_a_unir:
    # Llamamos a la función para unir las palabras en el corpus actual
    corpus_modificado = unir_palabras_en_contexto(corpus_modificado, palabra1, palabra2, 1)
'''
    
for token in words:
    if token not in palabras_a_indice:
        index = len(palabras_a_indice)
        palabras_a_indice[token] = index
        indices_a_palabras[index] = token
        diccionario_conteo[token] = 1 # Initialize count for new token
    else:
        diccionario_conteo[token] += 1 # Increment count for existing token


cardinal_V = len(palabras_a_indice)

for token, idx in list(palabras_a_indice.items()):

    one_hot_vector = np.zeros(cardinal_V)
    one_hot_vector[idx] = 1
    diccionario_onehot[token] = one_hot_vector
    diccionario_onehot_a_palabra[str(one_hot_vector)] = token

In [41]:
len(words)

55480

In [42]:
len(set(words))

7757

In [113]:
len(set(corpus_modificado))

28007

In [114]:
len(corpus_modificado)

294868

In [46]:
def cargar_modelo_completo(nombre_archivo='pesos_cbow_pc2_epoca0.npz'):
    """
    Carga los pesos W1, W2 y los hiperparámetros N, C y eta
    desde un archivo .npz.
    """
    try:
        data = np.load(nombre_archivo)

        W1 = data['W1']
        W2 = data['W2']

        N = data['N'].item()
        C = data['C'].item()
        eta = data['eta'].item()

        print()

        return W1, W2,N, C, eta

    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{nombre_archivo}'.")
        return None, None, None, None, None


In [53]:
W1, W2,N, C, eta = cargar_modelo_completo("C:\\Users\\User\\Documents\\GitHub\\Aprendizaje_Automatico\\pesos_cbow_neg_epoca1300_contexto_5.npz")


In [48]:
C

5

In [50]:
def generar_tuplas_nuevo(corpus, palabras_a_indice, contexto):

    indices = [i for i in range(contexto,(len(corpus)-contexto))]

    indices_contexto = [i for i in range(-contexto,0)] + [i for i in range(1,contexto+1)]

    indices_tuplas = []

    contexto_a_central = {} 

    for i in indices:

        palabra_central = palabras_a_indice[corpus[i]]

        contexto_actual = tuple(palabras_a_indice[corpus[i+j]] for j in indices_contexto)

        if contexto_actual in contexto_a_central:
            if contexto_a_central[contexto_actual] != palabra_central:
                continue 
        else:
            contexto_a_central[contexto_actual] = palabra_central

        indices_tuplas.append((palabra_central, list(contexto_actual)))

    return indices_tuplas


len(generar_tuplas_nuevo(words,palabras_a_indice,5))

55469

In [44]:
def generar_tuplas(corpus, palabras_a_indice, contexto):

    ## Genero una lista de todos los indices de las palabras en el corpus, posibles sin padding
    indices = [i for i in range(contexto,(len(corpus)-contexto))]

    ## Genero una lista de los contextos de las palabras de contexto
    indices_contexto = [i for i in range(-contexto,0)] + [i for i in range(1,contexto+1)]

    indices_tuplas = []

    ## Por cada indice en indices, genero una tupla (indice_central, [indices_contexto])
    for i in indices:

        indices_tuplas.append((palabras_a_indice[corpus[i]], [palabras_a_indice[corpus[i+j]] for j in indices_contexto]))

    return indices_tuplas   

len(generar_tuplas(words,palabras_a_indice,5))

55470

In [43]:
#! pip install cupy-cuda11x


def softmax(x):
    x = np.asarray(x)
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def predecir_cbow(palabra_a_indices, indices_a_palabras, W1, W2, contexto_indices, topk=5):

    h = np.mean(W1[contexto_indices], axis=0).reshape(-1, 1)

    u = W2.T @ h
    #y = sigmoid(u).flatten()
    y = softmax(u).flatten()
    # --- Opción alternativa: similitud coseno ---
    # Normalizamos vectores de palabras y contexto
    #W2_norm = W2 / np.linalg.norm(W2, axis=0, keepdims=True)
    #h_norm = h / np.linalg.norm(h)
    #similitudes = (W2_norm.T @ h_norm).flatten()
    #candidatos = np.argsort(-similitudes)

    candidatos = np.argsort(-y)
    palabra_top1 = indices_a_palabras[candidatos[0]]
    palabras_topk = [indices_a_palabras[i] for i in candidatos[:topk]]

    return palabra_top1, palabras_topk


def predecir_cbow_completo(palabra_a_indices, indices_a_palabras, corpus, W1, W2, contexto, topk=5):
    
    tuplas = generar_tuplas_nuevo(corpus, palabra_a_indices, contexto)
    aciertos_top1 = 0
    aciertos_topk = 0
    palabras_mal_predichas = []

    totales = len(tuplas)

    for i, (p_objetivo, p_contexto) in enumerate(tuplas):
        palabra_top1, palabras_topk = predecir_cbow(
            palabra_a_indices, indices_a_palabras, W1, W2, p_contexto, topk=topk
        )

        # Top-1
        if indices_a_palabras[p_objetivo] == palabra_top1:
            aciertos_top1 += 1

        # Top-k
        if indices_a_palabras[p_objetivo] in palabras_topk:
            aciertos_topk += 1
        else:
            palabras_mal_predichas.append(indices_a_palabras[p_objetivo])

        # Progreso
        if i % 100 == 0 and i > 0:
            progreso = (i / totales) * 100
            print(f"Progreso: [{progreso:.1f}%] - Top1: {aciertos_top1/totales:.4f}%, Top{topk}: {aciertos_topk/totales:.4f}%", end='\r')

    accuracy_top1 = aciertos_top1 / totales
    accuracy_topk = aciertos_topk / totales

    print(f"\nResultados completos: Top1={accuracy_top1*100:.2f}%, Top{topk}={accuracy_topk*100:.2f}%")

    return accuracy_top1, accuracy_topk, palabras_mal_predichas

In [55]:
accuracy_top1, accuracy_topk, palabras_mal_predichas = predecir_cbow_completo(palabras_a_indice,indices_a_palabras,words,W1,W2,5,5)


Progreso: [99.9%] - Top1: 0.0181%, Top5: 0.3894%
Resultados completos: Top1=1.81%, Top5=38.99%


In [96]:
print(f"Accuracy flexible: {accuracy_topk}")
print(f"Error flexible: {1 - accuracy_topk}")
print()
print(f"Accuracy: {accuracy_top1}")
print(f"Error: {1 - accuracy_top1}")

Accuracy flexible: 0.11581910552177896
Error flexible: 0.884180894478221

Accuracy: 0.024602315316065628
Error: 0.9753976846839344


In [4]:
lista = words
len(set(lista))

7755

In [5]:
cuenta = {}

for i in lista:
    if i not in cuenta:
        cuenta[i] = 1
    else:
        cuenta[i] += 1


In [6]:
import pandas as pd

cuenta = pd.Series(cuenta)
cuenta = cuenta.sort_values(ascending=False)
pd.set_option('display.max_rows', 200)  
cuenta.head()


,      3382
de     2258
que    1905
la     1868
y      1868
dtype: int64

In [8]:



def palabras_frecuentes_en_contexto(corpus, palabra_objetivo, contexto=1):

    frecuencias = {}  
    
    for i in range(len(corpus)):
        
        if corpus[i] == palabra_objetivo:
            
            for j in range(i - contexto, i + contexto + 1):
                
                if j != i and 0 <= j < len(corpus):  
                    palabra_contexto = corpus[j]
                    
                    if palabra_contexto in frecuencias:
                        frecuencias[palabra_contexto] += 1
                    else:
                        frecuencias[palabra_contexto] = 1
    
    
    return sorted(frecuencias.items(), key=lambda x: x[1], reverse=True)


#corpus_modificado = palabras_mal_predichas.copy()

for palabra in cuenta.index:
    if cuenta[palabra] > 500:
        palabra_objetivo = palabra
        resultados = palabras_frecuentes_en_contexto(words, palabra_objetivo, 1)
        print(f"Palabras más frecuentes cerca de '{palabra_objetivo}',que fue aparece:{cuenta[palabra]}\n")
        for palabra, frecuencia in resultados[:5]:
            #corpus_modificado = unir_palabras_en_contexto(corpus_modificado, palabra_objetivo, palabra, 1)
            print(f"{palabra}: {frecuencia}")
            
        print("\n" + "="*50 + "\n")

Palabras más frecuentes cerca de ',',que fue aparece:3382

y: 468
pero: 183
el: 126
que: 126
no: 98


Palabras más frecuentes cerca de 'de',que fue aparece:2258

la: 318
los: 123
que: 111
las: 106
un: 67


Palabras más frecuentes cerca de 'que',que fue aparece:1905

lo: 138
no: 131
,: 126
se: 119
de: 111


Palabras más frecuentes cerca de 'la',que fue aparece:1868

de: 318
en: 201
a: 188
,: 89
que: 77


Palabras más frecuentes cerca de 'y',que fue aparece:1868

,: 468
el: 123
la: 72
que: 71
se: 68


Palabras más frecuentes cerca de '.',que fue aparece:1495

.: 238
el: 76
no: 64
pero: 59
a: 49


Palabras más frecuentes cerca de 'el',que fue aparece:1463

en: 184
,: 126
y: 123
que: 104
con: 79


Palabras más frecuentes cerca de 'a',que fue aparece:1434

la: 188
,: 75
su: 63
y: 59
.: 49


Palabras más frecuentes cerca de 'en',que fue aparece:1040

la: 201
el: 184
que: 64
un: 52
los: 47


Palabras más frecuentes cerca de 'se',que fue aparece:819

que: 119
no: 70
y: 68
,: 60
había: 38


Pal

In [10]:
import pandas as pd

# ==========================
# 1. Funciones auxiliares
# ==========================

def palabras_frecuentes_en_contexto(corpus, palabra_objetivo, contexto=1):
    """Devuelve las palabras más frecuentes alrededor de una palabra objetivo."""
    frecuencias = {}  
    for i in range(len(corpus)):
        if corpus[i] == palabra_objetivo:
            for j in range(i - contexto, i + contexto + 1):
                if j != i and 0 <= j < len(corpus):
                    palabra_contexto = corpus[j]
                    frecuencias[palabra_contexto] = frecuencias.get(palabra_contexto, 0) + 1
    return sorted(frecuencias.items(), key=lambda x: x[1], reverse=True)


def unir_palabras_en_contexto(corpus, palabra1, palabra2):
    nuevo_corpus = []
    i = 0
    while i < len(corpus):
        # Caso: palabra1 seguida de palabra2
        if corpus[i] == palabra1 and i + 1 < len(corpus) and corpus[i + 1] == palabra2:
            nuevo_corpus.append(f"{palabra1} {palabra2}")
            i += 2  # saltamos ambas palabras ya unidas
        # Caso: palabra2 seguida de palabra1
        elif corpus[i] == palabra2 and i + 1 < len(corpus) and corpus[i + 1] == palabra1:
            nuevo_corpus.append(f"{palabra2} {palabra1}")
            i += 2
        else:
            nuevo_corpus.append(corpus[i])
            i += 1
    return nuevo_corpus


# ==========================
# 2. Cálculo de frecuencias
# ==========================

def contar_palabras(corpus):
    cuenta = {}
    for palabra in corpus:
        cuenta[palabra] = cuenta.get(palabra, 0) + 1
    cuenta = pd.Series(cuenta)
    cuenta = cuenta.sort_values(ascending=False)
    return cuenta


# ==========================
# 3. Bucle principal
# ==========================

def optimizar_corpus(words, min_frecuencia=200, contexto=1, top_contextos=3, iteraciones=3, frecuencia_min=100):
    corpus_modificado = words.copy()
    
    for iteracion in range(iteraciones):  # hacer varias pasadas para mejorar combinaciones
        print(f"\n====== Iteración {iteracion + 1} ======\n")

        cuenta = contar_palabras(corpus_modificado)
        palabras_objetivo = cuenta[cuenta > min_frecuencia].index

        for palabra_objetivo in palabras_objetivo:
            resultados = palabras_frecuentes_en_contexto(corpus_modificado, palabra_objetivo, contexto)
            
            print(f"Palabra '{palabra_objetivo}' (freq: {cuenta[palabra_objetivo]})")
            print("Contextos más frecuentes:")
            
            for palabra_contexto, frecuencia in resultados[:top_contextos]:
                if frecuencia > frecuencia_min:
                    print(f"  {palabra_contexto}: {frecuencia}")
                    corpus_modificado = unir_palabras_en_contexto(
                        corpus_modificado, palabra_objetivo, palabra_contexto)
            
            print("-" * 40)
        
        # recalcular frecuencias después de cada iteración
        cuenta = contar_palabras(corpus_modificado)
        print("\nTop 10 palabras tras iteración:\n", cuenta.head(10))
    
    return corpus_modificado

In [ ]:

corpus_modificado = optimizar_corpus(words, min_frecuencia=500, contexto=1, top_contextos=8,iteraciones=3, frecuencia_min=300)




====== Iteración 1 ======

Palabra ',' (freq: 14317)
Contextos más frecuentes:
  se: 373
  a: 359
  un: 346
  con: 313
----------------------------------------
Palabra 'de' (freq: 9139)
Contextos más frecuentes:
  un: 375
----------------------------------------
Palabra '.' (freq: 8569)
Contextos más frecuentes:
  no: 391
  . .: 344
  pero: 321
  la: 318
  y: 314
  el: 310
----------------------------------------
Palabra 'y' (freq: 6911)
Contextos más frecuentes:
  el: 304
----------------------------------------
Palabra 'que' (freq: 6084)
Contextos más frecuentes:
  la: 318
----------------------------------------
Palabra 'a' (freq: 5890)
Contextos más frecuentes:
----------------------------------------
Palabra 'la' (freq: 5805)
Contextos más frecuentes:
----------------------------------------
Palabra 'el' (freq: 4946)
Contextos más frecuentes:
----------------------------------------
Palabra 'un' (freq: 4148)
Contextos más frecuentes:
  en: 312
------------------------------------

In [31]:
corpus_modificado2 = corpus_modificado.copy()
corpus_modificado2 = optimizar_corpus(corpus_modificado, min_frecuencia=500, contexto=1, top_contextos=8,iteraciones=3, frecuencia_min=200)


====== Iteración 1 ======

Palabra ',' (freq: 12930)
Contextos más frecuentes:
  una: 279
  los: 265
  por: 241
  es: 232
  como: 222
----------------------------------------
Palabra 'de' (freq: 8764)
Contextos más frecuentes:
----------------------------------------
Palabra '.' (freq: 6649)
Contextos más frecuentes:
  a: 224
----------------------------------------
Palabra 'y' (freq: 6310)
Contextos más frecuentes:
  la: 285
  se: 272
----------------------------------------
Palabra 'que' (freq: 5766)
Contextos más frecuentes:
  en: 247
----------------------------------------
Palabra 'a' (freq: 5532)
Contextos más frecuentes:
  los: 229
  va: 209
----------------------------------------
Palabra 'la' (freq: 5175)
Contextos más frecuentes:
  maga: 266
  con: 265
  por: 257
----------------------------------------
Palabra 'el' (freq: 4339)
Contextos más frecuentes:
  por: 229
  con: 223
----------------------------------------
Palabra '. ' (freq: 3838)
Contextos más frecuentes:
  no: 2

In [32]:

len(corpus_modificado2)


284209

In [33]:

len(set(corpus_modificado2))


28061

In [34]:
with open("corpus_junto2.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(corpus_modificado))

In [35]:
corpus_modificado2

['encontraría',
 'a la',
 'maga',
 '?',
 'tantas',
 'veces',
 'me',
 'había',
 'bastado',
 'asomarme',
 ',',
 'viniendo',
 'por la',
 'rue',
 'de',
 'seine',
 ',',
 'al',
 'arco',
 'que',
 'da',
 'al',
 'quai',
 'de',
 'conti',
 ', y',
 'apenas',
 'la',
 'luz',
 'de',
 'ceniza',
 'y',
 'olivo',
 'que',
 'flota',
 'sobre',
 'el',
 'río',
 'me',
 'dejaba',
 'distinguir',
 'las',
 'formas',
 ',',
 'ya',
 'su',
 'silueta',
 'delgada',
 'se',
 'inscribía',
 'en el',
 'pont',
 'des',
 'arts',
 ', a',
 'veces',
 'andando',
 'de un',
 'lado',
 'a',
 'otro',
 ', a',
 'veces',
 'detenida',
 'en el',
 'pretil',
 'de',
 'hierro',
 ',',
 'inclinada',
 'sobre',
 'el',
 'agua',
 '. y',
 'era',
 'tan',
 'natural',
 'cruzar',
 'la',
 'calle',
 ',',
 'subir',
 'los',
 'peldaños',
 'del',
 'puente',
 ',',
 'entrar',
 'en',
 'su',
 'delgada',
 'cintura',
 'y',
 'acercarme',
 'a la',
 'maga',
 'que',
 'sonreía',
 'sin',
 'sorpresa',
 ',',
 'convencida',
 'como',
 'yo',
 'de que',
 'un',
 'encuentro',
 'cas

In [110]:
contar_palabras(corpus_modificado)

,            14317
de            9139
.             8569
y             6911
que           6084
             ...  
pollini          1
maurizio         1
milongas         1
piaña            1
sebastián        1
Length: 28007, dtype: int64